In [1]:
from ultralytics import YOLO

In [14]:
pt_model = YOLO("runs/detect/yolov9e 0.995 0.801/weights/last.pt")  # load a pretrained model (recommended for training)

In [10]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('../../../data/images/image_5095.jpg')
im2 = Image.open('../../../data/images/image_5100.jpg')

In [15]:
result = pt_model.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result]
result = [tuple(zip(*r)) for r in result]  # list of tuple[box, conf] in each image in xyxy format


0: 928x1600 7 targets, 59.5ms
1: 928x1600 7 targets, 59.5ms
Speed: 7.5ms preprocess, 59.5ms inference, 2.0ms postprocess per image at shape (1, 3, 928, 1600)


In [16]:
# plot
for im, boxes in zip([im1, im2], result):
    im = im.copy()
    draw = ImageDraw.Draw(im)
    for (x1, y1, x2, y2), conf in boxes:
        draw.rectangle(xy=((x1, y1), (x2, y2)), outline='red')
        draw.text((x1, y1), text=f'{conf:.2f}', fill='red')
    im.show()